In [1]:
import preprocessing as preproc
import helpers
from tqdm import tqdm
import time
import text_representation as text_repr
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Embedding, Activation, Flatten, GlobalMaxPooling1D, LSTM
from keras.optimizers import Adam, SGD

from keras.callbacks import ModelCheckpoint
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
start_time = time.time()

full_dataset = True
glove_pretrained = True

train_data, y, test_data = helpers.get_processed_data(full_dataset=full_dataset)

num_words = text_repr.define_num_words(train_data, full_dataset)

# word_embeddings = dict()
# if glove_pretrained:
#     word_embeddings = text_repr.loadGloveModel("../data/twitter-datasets/glove.twitter.27B.200d.txt")

143847: num of words , 120000: will be used


In [3]:
tokenizer = text_repr.fit_tokenizer(train_data, num_words)

# embedding_matrix = text_repr.get_embedding_matrix(tokenizer, word_embeddings, num_words)
# print('embedding_matrix.shape:', embedding_matrix.shape)
# print(embedding_matrix[0][-1])

tweet_max_length = text_repr.define_tweet_max_len(train_data, full_dataset)

train_data_seq = tokenizer.texts_to_sequences(train_data)
test_data_seq = tokenizer.texts_to_sequences(test_data)
train_data_seq = pad_sequences(train_data_seq, maxlen=tweet_max_length)
test_data_seq = pad_sequences(test_data_seq, maxlen=tweet_max_length)

127: max sentence length found, 130: will be used


In [ ]:
# keras.layers.recurrent.LSTM(output_dim, init='glorot_uniform'

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=200, weights=[embedding_matrix], 
                    input_length=tweet_max_length, trainable=True))
model.add(LSTM(units=32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(units=32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train, X_test, y_train, y_test = train_test_split(train_data_seq, y, test_size=0.2, random_state=42)


In [ ]:
filepath_acc = "../data/intermediate/lstm1.weights.best.hdf5"
checkpoint_acc = ModelCheckpoint(filepath_acc, monitor='val_acc', verbose=2, save_best_only=True, mode='max')

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32, verbose=1)

In [4]:
best_model = load_model("../data/intermediate/lstm1.weights.best.hdf5")

In [5]:

scores = []
for i in range(1,4):
    X_train, X_test, y_train, y_test = train_test_split(train_data_seq, y, test_size=10000, random_state=i)
    score = best_model.evaluate(X_test,y_test,verbose=1)
    print("%s: %.2f%%" % (best_model.metrics_names[1], score[1]*100))
    scores.append(score[1])
print('Average score:', sum(scores)/len(scores))

10000/10000 [==============================] - 10s 997us/step
acc: 86.75%
10000/10000 [==============================] - 12s 1ms/step
acc: 86.80%
10000/10000 [==============================] - 11s 1ms/step
acc: 86.41%
Average score: 0.8665333333333334


In [6]:
helpers.predict_and_save(best_model, test_data_seq, 'CNN')

In [ ]:
84070